In [1]:
import os
import json
from tqdm import tqdm
from tensorflow import keras
from IPython.display import clear_output
from collections import Counter
import time
import itertools

import matplotlib.pyplot as plt
import numpy as np


from pathlib import Path

In [2]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, N3
from pprint import pprint
import requests
import xml.etree.ElementTree as ET

In [3]:
# Load the dictionary with the lookup results for each cell value in the tabular data
def load_json(data_json):
    with open(data_json) as json_file:
        return json.load(json_file)
    
def dbo_sparql_results(query_string):
    
    classes = list([])
    dbo_prefix = 'http://dbpedia.org/ontology/'
    
    
    sparql = SPARQLWrapper('https://dbpedia.org/sparql')
    sparql.setQuery(query_string)
    
    try:
        sparql.setReturnFormat(JSON)
        qres = sparql.query().convert()
        for entity_class in qres['results']['bindings']:
            if dbo_prefix in entity_class[list(qres['results']['bindings'][0].keys())[0]]['value']:
                candicate_class = entity_class[list(qres['results']['bindings'][0].keys())[0]]['value'].split(dbo_prefix)[1]
                classes.append(candicate_class)
    except:
        pass
    
    return classes

def get_dbo_subclass(superClass):
    
    query_string = f'''
    SELECT distinct ?subClass 
    WHERE {{ ?subClass rdfs:subClassOf dbo:{superClass}. }}
    '''
    return dbo_sparql_results(query_string)


def get_dbo_superclass(subclass):
    
    query_string = f'''
    SELECT distinct ?superClass 
    WHERE {{ dbo:{subclass} rdfs:subClassOf ?superClass . }}
    '''
    
    return dbo_sparql_results(query_string)


def get_dbo_superclasses(subclass):
    classes = list([])
    
    try:
        parent = get_dbo_superclass(subclass)
    except:
        return []
    
    while len(parent) > 0:
        classes.append(parent[0])
        parent = get_dbo_superclass(parent[0])
    return classes

Load two dictionaries from a previous run:
* the cell_values dictionary that has an instance of each cell value and the candidate entities (and their types) in descending rank of retrieval from the API
* the predicted classes for each file / column that is a target from the CTA task

We use the best version of the CTA which was the TFIDF approach

In [4]:
output_folder = "output\\"
# cnn_model_directory = os.getcwd()+'\\output\\cnn_models'


cell_values = load_json(output_folder+'cell_values.json')
predicted_classes = load_json(output_folder+'predicted_classes.json')

The we perform the CTA task with the following paramenters:
* threshold is the number of predicted classes we are willin to consider for the candidate entity to be in. For instance 1 means we only consider entities in the top1 predicted types for each column from the CTA task. A threshold of 2 means we are a bit more flexible and allow entities in the top 2 predicted classes, etc.

In [68]:
def predict_dbp_cell_entity(cell_values, threshold = 1, use_cta=True, use_hierarchy=False):

    dbp_prefix = 'http://dbpedia.org/resource/'

    pred_cell_value = dict()
    
    class_parents = dict()

    for cell_value in dict(itertools.islice(cell_values.items(), 2000000)):
    #     print(cell_values[cell_value])

        #first we derive the predicted classes for the columns the cell has been found in. We can only select the top class per column or set a threshold
        pred_cls = list()
        for location in cell_values[cell_value]['location']:
            for cls in predicted_classes[location[0]][str(location[1])]['class_without_hr'][:threshold]:
                pred_cls.append(cls[0])
        pred_cls = list(set(pred_cls))
        
#         print(pred_cls)       
        if use_hierarchy == True:
            pred_cls_with_hierarchy = list()
            for cls in pred_cls:
                pred_cls_with_hierarchy.append(cls)
                
                try:
                    superclasses = class_parents[cls]
                except:
                    superclasses = get_dbo_superclasses(cls)
                    class_parents[cls] = superclasses

                for parent in superclasses:
                    pred_cls_with_hierarchy.append(parent)
            
            pred_cls = list(set(pred_cls_with_hierarchy))
#         print(pred_cls_with_hierarchy)

        #next we iterate through the retrieved entities and select the first candidate entity whose class is in the candidate classes
        for candidate_entity in cell_values[cell_value]['candidate_entities']:
    #         print(candidate_entity, cell_values[cell_value]['candidate_entities'][candidate_entity])

            # check if any of the candidate entity's classes is in the predicted classes and if so selecte that as the entity. The entities are coming ordered 
            if use_cta == True:
                if len(list(set(pred_cls) & set(cell_values[cell_value]['candidate_entities'][candidate_entity]['candidate_classes'])))>0:
                    pred_cell_value[cell_value] = dbp_prefix+candidate_entity
                    break
            else:
                pred_cell_value[cell_value] = dbp_prefix+candidate_entity
                break
    return pred_cell_value
        

Delete the key "parsed_files" from the dictionary since it only has the list of files that have been processed to get the cell values instead of a cell value itself

In [69]:
try:
    del cell_values["parsed_files"]
except:
    pass

pred_cell_value = predict_dbp_cell_entity(cell_values, 1, True, True)

In [70]:
pred_cell_value['Pekan District']

'http://dbpedia.org/resource/Pekan_District'

With candidates selected of the set of cell values the last step is to parse the files once again and construct the output for the column results

In [58]:
"""
lookup candidate entities and classes
"""
# import os
import pandas as pd
# import sys
import argparse
# import pyspark
# from pyspark.sql import SparkSession
# from pyspark import SparkConf, SparkContext
# import json
# import time
from itertools import islice
# from collections import Counter
# from tqdm import tqdm
# import re

current_path = os.getcwd()
parser = argparse.ArgumentParser()
parser.add_argument(
    '--input_dir',
    type=str,
    default=os.path.join(current_path, 'data'),
    help='Directory of input/output')
parser.add_argument(
    '--dataset',
    type=str,
#     default='round_1',
    default='2020_2T',
    help='The folder containing the input data')
parser.add_argument(
    '--target_filename',
    type=str,
#     default='CEA_Round1_Targets.csv',
    default='CEA_2T_Targets.csv',    
    help='The name of the file that contains the target types for each column')
parser.add_argument(
    '--gt_filename',
    type=str,
#     default='CEA_Round1_gt.csv',
    default='CEA_2T_gt.csv',
    help='The name of the file that contains the ground truth for each column')
parser.add_argument(
    '--file_type',
    type=str,
    default='csv',
    help='File type')


FLAGS, unparsed = parser.parse_known_args()
# if not os.path.exists(FLAGS.input_dir):
#     os.mkdir(FLAGS.input_dir)

In [59]:
# Get all the csv files from the input directory
def get_data_files(data_folder):
    """
    A function used to get all the csv files from the input directory
    ...

    Attributes
    ----------
    data_folder : str
        the folder within  the working directory where the data is located
    """

    files = [] # a list of all filenames, including file extensions, that contain data
    csv_files = [] # same list as above but without the file extension

    # Get the list of files
    files = [f for f in os.listdir(FLAGS.input_dir+data_folder) if os.path.isfile(os.path.join(FLAGS.input_dir+data_folder, f))]
    csv_files = [f.replace(".csv","") for f in os.listdir(FLAGS.input_dir+data_folder) if os.path.isfile(os.path.join(FLAGS.input_dir+data_folder, f))]
    
    return csv_files

def get_target_cea_column_cells(target_config_file, data_folder, csv_files, filter_col = True):
    """
    A function used to get which columns from the csv files need to be considered for the CTA. This is a subset of the file columns ignoring anything that is not an entity
    ...

    Attributes
    ----------
    target_config_file : str
        the file that contains the target column indices for each file
    csv_files : list
        the list of csv files that have the tabular data
    filter_col : boolean
        a flag to indicate whether we should narrow down the reading of the columns to only those targeted for the CTA task
    """
   
    target_col_file = os.path.join(FLAGS.input_dir+data_folder, target_config_file)
    df_target_col = pd.read_csv(target_col_file,header=None, names=['filename','column_index', 'cell_index'])
    
    # filter to only those files that are included in the csv_files
    df_target_col = df_target_col.loc[df_target_col['filename'].isin(csv_files)]
    
    # collapse all rows pertaining to the same file into one key value pair. The key is the filename and the value is the list with the column indices that should be considered
    # dict_target = {'CTRL_DBP_GEO_european_countries_capital_populated_cities': [0, 1, 2]}
    dict_target = dict()
    
    for index,row in df_target_col.iterrows():
#         print(row)
        
        # is this is the first row with this file create the key
        if row['filename'] not in dict_target:
            dict_target[row['filename']]= dict()
        
        if row['column_index'] not in dict_target[row['filename']]:
            dict_target[row['filename']][row['column_index']]= list()
        # append the new target column to the target column list for that file
        if filter_col:
            dict_target[row['filename']][row['column_index']].append(int(row['cell_index']))

        dict_target[row['filename']][row['column_index']].sort()
    return dict_target

def get_ground_truth(file, folder, csv_files):
    """
    A function used to get the ground truths as provided in the setup
    ...

    Attributes
    ----------
    file : str
        the file that contains the ground truth for the class of each column in each file
    folder : str
        the folder that contains the ground truth file
    csv_files : list
        the list of csv files that have the tabular data
    """
    
    dbo_prefix = 'http://dbpedia.org/ontology/'
   
    filepath = os.path.join(FLAGS.input_dir+folder, file)
    df_ground_truth = pd.read_csv(filepath,header=None, names=['filename','column_index', 'cell_index', 'entity'])
    
    # filter to only those files that are included in the csv_files
    df_ground_truth = df_ground_truth.loc[df_ground_truth['filename'].isin(csv_files)]
    
    # collapse all rows pertaining to the same file into one key value pair. The key is the filename and the value is the list with the column indices that should be considered
    # dict_target = {'CTRL_DBP_GEO_european_countries_capital_populated_cities': [0, 1, 2]}
    dict_gt = dict()
    
    for index,row in df_ground_truth.iterrows():
        
        # is this is the first row with this file create the key
        if row['filename'] not in dict_gt:
            dict_gt[row['filename']]= dict()
        if row['column_index'] not in dict_gt[row['filename']]:
            dict_gt[row['filename']][row['column_index']]= dict()
        
        dict_gt[row['filename']][row['column_index']][int(row['cell_index'])] = row['entity']
            
    return dict_gt

def read_data(data_folder, dict_target_col, has_header_row = False):
    """
    A function used to read the data from the csvs in the data_folder only considering the columns that are in the dict_target_col
    ...

    Attributes
    ----------
    folder : str
        the folder that contains the csvs with the tabular data
    dict_target_col : dictionary
        a dictionary with csv filenames as the key and an array of relevant column indices as a value
    has_header_row : boolean
        a flag to indicate whether the first row in the csv files needs to be skipped as it is a header
    """
    temp_list = list()

    for file in dict_target_cell:

        filename = file + '.' + FLAGS.file_type
        tab_data_file = os.path.join(FLAGS.input_dir + data_folder, filename)
        
        df_data = pd.read_csv(tab_data_file,header=None, skiprows=[0], usecols=dict_target_col[file])
        
        for index,row in df_data.iterrows():
            for col in dict_target_cell[file]:
                if index+1 in dict_target_cell[file][col]:
                    temp_list.append([file, col, index+1, row[col]])
                    
    
    return pd.DataFrame(temp_list, columns=['filename', 'column', 'cell', 'cell_value'])

In [60]:
# Get the list of csv files with tabular data
csv_files = get_data_files('\\%s\\tables' % FLAGS.dataset)
# csv_files = csv_files[:1]
csv_files[:5]

['CTRL_DBP_BUS_automobile_manufacturer',
 'CTRL_DBP_BUS_automobile_manufacturer_NOISE2',
 'CTRL_DBP_BUS_european_company_high_revenues',
 'CTRL_DBP_BUS_european_company_high_revenues_NOISE2',
 'CTRL_DBP_BUS_videogame_publishers']

In [61]:
# Get the columns we need to consider for the CTA task
dict_target_cell = get_target_cea_column_cells(FLAGS.target_filename, '\\%s\\targets' % FLAGS.dataset, csv_files,True)
# list(islice(dict_target_col.items(), 5))

In [62]:
ground_truth = get_ground_truth(FLAGS.gt_filename, '\\%s\\gt' % FLAGS.dataset, csv_files)
# list(islice(ground_truth.items(), 1))

Read the data from the files and load in a dataframe with the col and cell indicator

In [63]:
data = read_data('\\%s\\tables' % FLAGS.dataset, dict_target_cell,True)

...and enrich with the prediction:
* taking into account only entities of the suggected cta predicted class
* selecting the top entity retrieved by the lookupg
* selected the entity considering the cta and if no entity is in the cta classes default to the top one retieved

In [130]:
def allocate_entity(cell_value):
    try:
        return pred_cell_value[cell_value]
    except: pass
    
pred_cell_value = predict_dbp_cell_entity(cell_values, 1, True, True)
data['pred_entity_with_cta'] = data.apply (lambda row: allocate_entity(row.cell_value), axis=1)

pred_cell_value = predict_dbp_cell_entity(cell_values, 1, False)
data['pred_entity_without_cta'] = data.apply (lambda row: allocate_entity(row.cell_value), axis=1)

pred_cell_value = predict_dbp_cell_entity(cell_values, 1, True, False)
data['pred_entity_with_cta_withoutpartents'] = data.apply (lambda row: allocate_entity(row.cell_value), axis=1)

data['pred_entity_hybrid'] = data.apply(lambda row: row.pred_entity_with_cta if row.pred_entity_with_cta_withoutpartents is None else row.pred_entity_with_cta, axis = 1)

In [131]:
pred_cell_value = predict_dbp_cell_entity(cell_values, 1, True, False)
data['pred_entity_with_cta_withoutpartents'] = data.apply (lambda row: allocate_entity(row.cell_value), axis=1)

In [132]:
# pred_cell_value['Pekan District']

...and enrich with expected result

In [133]:
data['expected_entity'] = data.apply (lambda row: ground_truth[row.filename][row.column][row.cell], axis=1)

In [134]:
def evaluate_cea(data, approach):
    total = data.shape[0]
    total_annotated = 0
    found = 0

    for index, row in data.iterrows():
        try:
            if row[approach] in row.expected_entity:
                found += 1
            total_annotated +=1
        except:
            pass

    precision = found*100 / total_annotated
    recall = found*100 / total
    f1_score = (2 * precision * recall) / (precision + recall)

    return(precision, recall, f1_score)

(precision, recall, f1_score) = evaluate_cea(data, 'pred_entity_without_cta')
print(f"Precision: {precision}, recall: {recall} and f1_score: {f1_score}")

(precision, recall, f1_score) = evaluate_cea(data, 'pred_entity_with_cta')
print(f"Precision: {precision}, recall: {recall} and f1_score: {f1_score}")

(precision, recall, f1_score) = evaluate_cea(data, 'pred_entity_with_cta_withoutpartents')
print(f"Precision: {precision}, recall: {recall} and f1_score: {f1_score}")


(precision, recall, f1_score) = evaluate_cea(data, 'pred_entity_hybrid')
print(f"Precision: {precision}, recall: {recall} and f1_score: {f1_score}")


Precision: 77.34104677542929, recall: 70.42534083922996 and f1_score: 73.72136069572677
Precision: 81.05828480212116, recall: 72.22958882312524 and f1_score: 76.38969011049954
Precision: 84.17090016750188, recall: 73.59791819858481 and f1_score: 78.53013148459581
Precision: 81.05828480212116, recall: 72.22958882312524 and f1_score: 76.38969011049954


In [ ]:
data[['filename', 'column', 'cell', 'pred_entity_hybrid']].to_csv('output/cea_predictions-%s.csv' % time.strftime("%Y%m%d-%H%M%S"), index = False, header = False)

# ('output/cea_predictions-%s.csv' % time.strftime("%Y%m%d-%H%M%S")

In [106]:
is_NaN = data.isnull()
row_has_NaN = pd.DataFrame(is_NaN['pred_entity_without_cta']).any(axis=1)
rows_with_NaN = data[row_has_NaN]

In [118]:
is_NaN = rows_with_NaN.isnull()
row_has_both_NaN = pd.DataFrame(is_NaN['pred_entity_with_cta_withoutpartents']).any(axis=1)
rows_with_both_NaN = data[row_has_NaN]

In [123]:
rows_with_both_NaN.shape

(59343, 9)